In [132]:
import random
from copy import deepcopy
from networkx import Graph, minimum_spanning_tree
import networkx as nx

class Individual:
    def __init__(self, edges):
        self.edges = edges
        self.initialize_spanning_tree()
        self.fitness = 0
        self.calc_fitness()
    
    def initialize_spanning_tree(self):
        G = Graph()
        G.add_edges_from(self.edges)
        mst = minimum_spanning_tree(G)
        self.edges = deepcopy(list(mst.edges))
    
    def calc_fitness(self):
        G = Graph()
        G.add_edges_from(self.edges)
        self.fitness = sum(1 for node in G if G.degree[node] == 1)


In [147]:
def crossover(parent_1, parent_2):

    combined_edges = set()
    for u, v in parent_1.edges:
        if u < v:
            combined_edges.add((u, v))
        else :
            combined_edges.add((v, u))

    return Individual(list(combined_edges))

In [134]:
def replace_edge(tree_edges, all_edges):
    for _ in range(20):
        u, v = random.choice(all_edges)
        if not ((u, v) in tree_edges or (v, u) in tree_edges):
            tree_edges[random.randint(0, len(tree_edges) - 1)] = (u, v)
            break
    return tree_edges

In [135]:
def is_feasible(edges):
    G = Graph();
    G.add_edges_from(list(edges))
    return nx.is_connected(G)

In [136]:
def mutation(all_edges, individual, mutation_prob):
    if random.random() < mutation_prob:
        edges =  replace_edge(individual.edges, all_edges)
        if is_feasible(edges):
            individual.edges = deepcopy(edges)
            individual.initialize_spanning_tree()
            individual.calc_fitness()

In [137]:
def selection(population, tournament_size):
      participants = random.sample(population, tournament_size)
      return max(participants, key=lambda x: x.fitness)

In [146]:
def simulated_annealing(individual, num_iters, all_edges):
    
    best_edges = deepcopy(individual.edges)
    best_fitness = individual.fitness
    edges = deepcopy(individual.edges)
    fitness = individual.fitness

    for it in range(2, num_iters + 2):
        for i in range(len(edges)):
            new_edges = deepcopy(replace_edge(edges, all_edges))

            G = Graph()
            G.add_edges_from(new_edges)
            new_fitness = sum(1 for node in G.nodes if G.degree[node] == 1)

            if new_fitness > fitness:
                #edges = new_edges
                fitness = new_fitness
                if new_fitness > best_fitness:
                    best_edges = deepcopy(new_edges)
                    best_fitness = new_fitness
                break
            else:
                p = random.random()
                q = 1 / it
                if p < q:
                    edges = new_edges
                    fitness = new_fitness

    best_edges_set = set(best_edges)
    if is_feasible(best_edges_set) and best_fitness > individual.fitness and len(best_edges_set) == len(individual.edges):
        #print("izvrsen SA")
        #print(individual)
        individual.edges = deepcopy(best_edges_set)
        individual.initialize_spanning_tree()
        individual.calc_fitness()
        #print(individual)
        
    
    return individual


In [139]:
def ga(graph, num_iters, sa_iters, elitism_size, mutation_prob, population_size, tournament_size):

    population = [Individual(list(graph.edges)) for _ in range(population_size)]
    initial_mst = population[0]
    new_population = deepcopy(population)
    
    if elitism_size % 2 != len(population) % 2:
        elitism_size += 1
    
    for _ in range(num_iters):
        population.sort(key=lambda x: x.fitness, reverse=True)
        new_population[:elitism_size] = deepcopy(population[:elitism_size])
    
        for i in range(elitism_size, population_size, 2):
            parent_1 = selection(population, tournament_size)
            parent_2 = selection(population, tournament_size)
            
            child_1 = crossover(parent_1, parent_2)
            child_2 = crossover(parent_1, parent_2)
            
            mutation(list(graph.edges), child_1, mutation_prob)
            mutation(list(graph.edges), child_2, mutation_prob)
            
            new_population[i] = simulated_annealing(child_1, sa_iters, list(graph.edges))
            new_population[i + 1] = simulated_annealing(child_2, sa_iters, list(graph.edges))
            
            population = deepcopy(new_population)

    feasible_population = []
    for p in population:
        G = Graph()
        G.add_edges_from(p.edges)
        if nx.is_connected(G):
            feasible_population.append(p)
    
    return initial_mst.fitness, max(feasible_population, default=initial_mst, key=lambda x: x.fitness)